In [1]:
compile () {
  gcc -o $1 -xc -Wall -Werror - -lrt
}

## Leaking a local

Valgrind detects a leaked local as definitely lost:

In [3]:
compile leak_local <<EOF
#include<stdlib.h>
int main(int argc, char **argv) {
  int *i = malloc(sizeof(*i));
  return 0;
}
EOF

In [5]:
valgrind ./leak_local

==26083== Memcheck, a memory error detector
==26083== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.
==26083== Using Valgrind-3.13.0 and LibVEX; rerun with -h for copyright info
==26083== Command: ./leak_local
==26083== 
==26083== 
==26083== HEAP SUMMARY:
==26083==     in use at exit: 4 bytes in 1 blocks
==26083==   total heap usage: 1 allocs, 0 frees, 4 bytes allocated
==26083== 
==26083== LEAK SUMMARY:
==26083==    definitely lost: 4 bytes in 1 blocks
==26083==    indirectly lost: 0 bytes in 0 blocks
==26083==      possibly lost: 0 bytes in 0 blocks
==26083==    still reachable: 0 bytes in 0 blocks
==26083==         suppressed: 0 bytes in 0 blocks
==26083== Rerun with --leak-check=full to see details of leaked memory
==26083== 
==26083== For counts of detected and suppressed errors, rerun with: -v
==26083== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)


## Leaking a global (not leaked)

A "leaked" global is still reachable at program exit, since the global is always in scope. Valgrind correctly detects this as still-allocated, but not a leak:

In [7]:
compile leak_global <<EOF

#include<stdlib.h>

static int *i;

int main(int argc, char **argv) {
  i = malloc(sizeof(*i));
  return 0;
}
EOF

In [8]:
valgrind ./leak_global

==26096== Memcheck, a memory error detector
==26096== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.
==26096== Using Valgrind-3.13.0 and LibVEX; rerun with -h for copyright info
==26096== Command: ./leak_global
==26096== 
==26096== 
==26096== HEAP SUMMARY:
==26096==     in use at exit: 4 bytes in 1 blocks
==26096==   total heap usage: 1 allocs, 0 frees, 4 bytes allocated
==26096== 
==26096== LEAK SUMMARY:
==26096==    definitely lost: 0 bytes in 0 blocks
==26096==    indirectly lost: 0 bytes in 0 blocks
==26096==      possibly lost: 0 bytes in 0 blocks
==26096==    still reachable: 4 bytes in 1 blocks
==26096==         suppressed: 0 bytes in 0 blocks
==26096== Rerun with --leak-check=full to see details of leaked memory
==26096== 
==26096== For counts of detected and suppressed errors, rerun with: -v
==26096== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)
